In [1]:
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source("cleanup.R")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [ ]:
url <- "http://roycekimmons.com/system/generate_data.php?dataset=exams&n=1000"
exam_data <- read_csv(url)
colnames(exam_data) <- c("gender", "race_ethnicity", "parental_level_of_education", "lunch", "test_preparation_course", 
                         "math_score", "reading_score", "writing_score")
exam_data1<- exam_data |>
mutate(parental_level_of_education = as_factor(parental_level_of_education)) |>
mutate("mean_score" = (math_score + reading_score + writing_score)/3) |>
select(parental_level_of_education, math_score, reading_score, writing_score, mean_score)

exam_data1

exam_graph <- exam_data1 |>
ggplot(aes(x = math_score, y = reading_score, color = parental_level_of_education)) +
geom_point(alpha = 0.5) +
labs(color = "Parental level of education")

exam_graph

#exam_graph <- exam_data1 |>
#ggplot(aes(x = mean_score, y = fct_reorder(parental_level_of_education, mean_score, .desc = TRUE), 
#           fill = parental_level_of_education)) +
#geom_bar(stat = "identity") + 
#labs(x = "Average Score", y = "Parental Level of Education",  fill = "type")

data_split <- initial_split(exam_data1, prop = 0.75, strata = parental_level_of_education)
data_train <- training(data_split)
data_test <- testing(data_split)

data_recipe <- recipe(parental_level_of_education ~ math_score + reading_score + writing_score, data = data_train) |>
step_scale(all_predictors()) |>
step_center(all_predictors())

In [ ]:
set.seed(1234) 
options(repr.plot.height = 5, repr.plot.width = 6)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
set_engine("kknn") |>
set_mode("classification")

k_vals <- tibble(neighbors = seq(from = 1, to = 10))
vfold <- vfold_cv(data_train, v = 5, strata = parental_level_of_education)
recipe <- recipe(parental_level_of_education ~., data = data_train)

knn_results <- workflow() |>
add_recipe(recipe) |>
add_model(knn_spec)|>
tune_grid(resamples = vfold, grid = k_vals) |>
collect_metrics()

accuracies <- knn_results |>
filter(.metric == "accuracy")

accuracies

cross_val_plot <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
geom_point() +
geom_line() +
labs(x = "Neighbors", y = "Accuracy Estimate")

cross_val_plot

In [ ]:
pivot_wider(exam_data1,
           names_from = parental_level_of_education,
           values_from = mean_score)

In [ ]:
group_by(exam_data1, parental_level_of_education) |>
summarize(below_bachelor = some)

In [ ]:
exam_data

In [ ]:
graph1 <- ggplot(data_train, aes(x = reading_score, y = writing_score)) +
geom_point() +
xlab("Reading Score") +
ylab("writing Score")

graph2 <- ggplot(data_train, aes(x = math_score, y = writing_score)) +
geom_point() +
xlab("Math Score") +
ylab("writing Score")

graph3 <- ggplot(data_train, aes(x = math_score, y = reading_score)) +
geom_point() +
xlab("Math Score") +
ylab("Reading Score")

graph1
graph2
graph3

exam_data1<- exam_data |>
mutate("mean_score" = (math_score + reading_score + writing_score)/3) |>
mutate(student = seq(from = 1, to = 1000)) |>
select(student, math_score, reading_score, writing_score, mean_score)

exam_data1

data_split <- initial_split(exam_data1, prop = 0.75, strata = reading_score)
data_train <- training(data_split)
data_test <- testing(data_split)

data_train
data_test
summary(data_train)

In [ ]:
data_recipe <- recipe(reading_score ~ writing_score, data = data_train) |>
 step_scale(all_predictors()) |>
 step_center(all_predictors())

data_spec <- nearest_neighbor(weight_func = "rectangular",
                             neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("regression")

data_vfold <- vfold_cv(data_train, v = 5, strata = reading_score)

data_wkflw <- workflow() |>
  add_recipe(data_recipe) |>
  add_model(data_spec)

data_wkflw

gridvals <- tibble(neighbors = seq(1, 200, by = 3))

data_results <- data_wkflw |>
tune_grid(resamples = data_vfold, grid = gridvals) |>
collect_metrics() |>
filter(.metric == "rmse")

data_results

data_multi <- workflow() |>
  add_recipe(data_recipe) |>
  add_model(data_spec) |>
  tune_grid(data_vfold, grid = gridvals) |>
  collect_metrics() |>
  filter(.metric == "rmse") |>
  filter(mean == min(mean))

data_k <- data_multi |>
              pull(neighbors)

data_multi

In [ ]:
data_spec <- nearest_neighbor(weight_func = "rectangular",  neighbors = 25) |>
set_engine("kknn")|>
set_mode("regression")

data_fit <- workflow() |>
add_recipe(data_recipe) |>
add_model(data_spec)|>
fit(data = data_train)

data_summary <- data_fit |>
predict(data_test) |>
bind_cols(data_test) |>
metrics(truth = reading_score, estimate = .pred) |>
filter(.metric == 'rmse')

data_summary

In [ ]:
data_preds <- tibble(writing_score = seq(1:100))

data_preds <- data_fit |>
predict(data_preds) |>
bind_cols(data_preds)

plot <- ggplot(data_train, aes(x = writing_score, y = reading_score)) +
geom_point() +
geom_line(data = data_preds,
         mapping = aes(x = writing_score, y = .pred),
         color = "blue") +
xlab("Writing Score") +
ylab("Reading Score") 

plot

In [ ]:
nearest_neighbors <- data_train |>
mutate(diff = abs(100 - writing_score)) |>
arrange(diff) |>
slice(1:5)

nearest_neighbors

In [ ]:
prediction <- nearest_neighbors |>
summarize(predicted = mean(reading_score))

prediction

In [ ]:
data_split <- initial_split(exam_data1, prop = 0.75, strata = reading_score)
data_train <- training(data_split)
data_test <- testing(data_split)

lm_spec <- linear_reg() |>
  set_engine("lm") |>
  set_mode("regression")

lm_recipe <- recipe(writing_score ~ reading_score, data = data_train)

lm_fit <- workflow() |>
  add_recipe(lm_recipe) |>
  add_model(lm_spec) |>
  fit(data = data_train)

lm_fit